# Out Paint

In [1]:
from keras.layers.convolutional import Conv2D, AtrousConvolution2D
from keras.layers import Activation, Dense, Input, Conv2DTranspose, Dense, Flatten
from keras.layers import ReLU, Dropout, Concatenate, BatchNormalization, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.convolutional import UpSampling2D
import os
import numpy as np
import PIL
import IPython.display

from keras_contrib.layers.normalization import InstanceNormalization

from dataloader import Data

Using TensorFlow backend.


In [2]:
# Initialize dataloader
data = Data()

In [4]:
INPUT_SHAPE = (256, 256, 3)
EPOCHS = 10000
BATCH = 15

# 25% i.e 64 width size will be mask from both side
MASK_PERCENTAGE = .25


CHECKPOINT = "checkpoint/"
SAVED_IMAGES = "saved_images/"

## Models

### Discriminator

In [5]:
d_input_shape = (INPUT_SHAPE[0], int(INPUT_SHAPE[1] * (MASK_PERCENTAGE *2)), INPUT_SHAPE[2])
d_dropout = 0.5
DCRM_OPTIMIZER = Adam(0.0001, 0.5)

(256, 128, 3)


In [6]:
def d_build_conv(layer_input, filter_size, kernel_size=4, strides=2, activation='leakyrelu', dropout_rate=d_dropout, norm=True):
    c = Conv2D(filter_size, kernel_size=kernel_size, strides=strides, padding='same')(layer_input)
    if activation == 'leakyrelu':
        c = LeakyReLU(alpha=0.2)(c)
    if dropout_rate:
        c = Dropout(dropout_rate)(c)
    if norm == 'inst':
        c = InstanceNormalization()(c)
    return c

def build_discriminator():
    d_input = Input(shape=d_input_shape)
    d = d_build_conv(d_input, 64, 5,strides=2, norm=False)
    for i in range(4):
        filter_size = 128 * (2**i)
        d = d_build_conv(d, 128, 5, strides=2)
    flat = Flatten()(d)
    fc1 = Dense(512, activation='relu')(flat)
    d_output = Dense(1, activation='sigmoid')(fc1)
    
    return Model(d_input, d_output)

In [7]:
DCRM = build_discriminator()
DCRM.summary()
DCRM.compile(loss='mse', optimizer=DCRM_OPTIMIZER, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 64, 64)       4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 64, 64)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 64, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 32, 128)       204928    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 32, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 32, 128)       0         
__________

### Generator Model

In [8]:
g_dropout = 0.5
GEN_OPTIMIZER = Adam(0.001, 0.5)

In [9]:
def g_build_conv(layer_input, filter_size, kernel_size=4, strides=2, activation='leakyrelu', dropout_rate=g_dropout, norm='inst', dilation=1):
    c = AtrousConvolution2D(filter_size, kernel_size=kernel_size, strides=strides,atrous_rate=(dilation,dilation), padding='same')(layer_input)
    if activation == 'leakyrelu':
        c = ReLU()(c)
    if dropout_rate:
        c = Dropout(dropout_rate)(c)
    if norm == 'inst':
        c = InstanceNormalization()(c)
    return c

def g_build_deconv(layer_input, filter_size, kernel_size=3, strides=2, activation='relu', dropout=0):
    d = Conv2DTranspose(filter_size, kernel_size=kernel_size, strides=strides, padding='same')(layer_input)
    if activation == 'relu':
        d = ReLU()(d)
    return d

def build_generator():
    g_input = Input(shape=INPUT_SHAPE)
    
    g1 = g_build_conv(g_input, 128, 5, strides=1)
    g2 = g_build_conv(g1, 256, 3, strides=2)
    g3 = g_build_conv(g2, 512, 3, strides=1)
    g4 = g_build_conv(g3, 512, 3, strides=1, dilation=2)
    g5 = g_build_conv(g4, 512, 3, strides=1, dilation=4)
    g6 = g_build_conv(g5, 512, 3, strides=1, dilation=8)
    g7 = g_build_conv(g6, 512, 3, strides=1)
    g8 = g_build_deconv(g7, 256, 4, strides=2)
    g9 = g_build_conv(g8, 128, 3, strides=1)
    
    g_output = AtrousConvolution2D(3, kernel_size=3, strides=(1,2), activation='tanh',padding='same', atrous_rate=(1,1))(g9)
    
    return Model(g_input, g_output)

In [10]:
# Generator Initialization
GEN = build_generator()
GEN.summary()
GEN.compile(loss='mse', optimizer=GEN_OPTIMIZER, metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:301: UserWarning: The `AtrousConvolution2D` layer  has been deprecated. Use instead the `Conv2D` layer with the `dilation_rate` argument.
  warnings.warn('The `AtrousConvolution2D` layer '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 256, 256, 128)     9728      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256, 256, 128)     0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256, 256, 128)     0         
_________________________________________________________________
instance_normalization_1 (In (None, 256, 256, 128)     2         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 256)     295168    
_________________________________________________________________
re_lu_2 (ReLU)               (None, 128, 128, 256)     0         
__________

### Combined Model

In [ ]:
IMAGE = Input(shape=INPUT_SHAPE)
DCRM.trainable = False
GENERATED_IMAGE = GEN(IMAGE)
CONF_GENERATED_IMAGE = DCRM(GENERATED_IMAGE)


COMBINED = Model(IMAGE, [CONF_GENERATED_IMAGE, GENERATED_IMAGE])
COMBINED.compile(loss=['mse', 'mse'], optimizer=GEN_OPTIMIZER)

### Masking and De-Masking

In [ ]:
def mask_width(img):
    image = img.copy()
    height = image.shape[0]
    width = image.shape[1]
    new_width = int(width * MASK_PERCENTAGE)
    mask = np.ones([height, new_width, 3])
    missing_x = img[:, :new_width]
    missing_y = img[:, width - new_width:]
    missing_part = np.concatenate((missing_x, missing_y), axis=1)
    image[:, :new_width] = mask
    image[:, width - new_width:] = mask
    return image, missing_part

def get_masked_images(images):
    mask_images = []
    missing_images = []
    for image in images:
        mask_image, missing_image = mask_width(image)
        mask_images.append(mask_image)
        missing_images.append(missing_image)
    return np.array(mask_images), np.array(missing_images)

def get_demask_images(original_images, generated_images):
    demask_images = []
    for o_image, g_image in zip(original_images, generated_images):
        width = g_image.shape[1] // 2
        x_image = g_image[:, :width]
        y_image = g_image[:, width:]
        o_image[:, :width] = x_image
        o_image[:, o_image.shape[1] - width:] = y_image
        demask_images.append(o_image)
    return np.asarray(demask_images)

### Utilities
1. Save Model
2. Load Model
3. Save Image

In [ ]:
def save_model():
    global DCRM, GEN
    models = [DCRM, GEN]
    model_names = ['DCRM','GEN']

    for model, model_name in zip(models, model_names):
        model_path =  CHECKPOINT + "%s.json" % model_name
        weights_path = CHECKPOINT + "/%s.hdf5" % model_name
        options = {"file_arch": model_path, 
                    "file_weight": weights_path}
        json_string = model.to_json()
        open(options['file_arch'], 'w').write(json_string)
        model.save_weights(options['file_weight'])
    print("Saved Model")
    
def load_model():
    global DCRM, GEN
    
    # load DCRM Model
    model_path = CHECKPOINT + "%s.json" % 'DCRM'
    weight_path = CHECKPOINT + "%s.hdf5" % 'DCRM'
    with open(model_path, 'r') as f:
        DCRM = model_from_json(f.read())
    DCRM.load_weights(weight_path)
    DCRM.compile(loss='mse', optimizer=d_optimizer, metrics=['accuracy'])
    
    #load GEN Model
    model_path = CHECKPOINT + "%s.json" % 'GEN'
    weight_path = CHECKPOINT + "%s.hdf5" % 'GEN'
    with open(model_path, 'r') as f:
         GEN = model_from_json(f.read())
    GEN.load_weights(weight_path
    
    print("loaded model")
    
    
import cv2    
def save_image(epoch, steps):
    mask, original = data.get_data(1)
    if mask is None:
        mask, original = data.get_data(1)
    
    mask_image, missing_image = get_masked_images(original)
    mask_image = mask_image / 127.5 - 1
    missing_image = missing_image / 127.5 - 1
    gen_missing = GEN.predict(mask_image)
    demask_image = get_demask_images(mask_image, gen_missing)
    
    demask_image = (demask_image + 1) * 127.5
    demask_image = demask_image.astype(np.uint8)
    mask_image = (mask_image + 1) * 127.5
    mask_image = mask_image.astype(np.uint8)
    
    file_name = str(epoch) + "_" + str(steps) + ".jpg"
    final_image = np.concatenate((original[0],mask_image[0], demask_image[0]), axis=1)
    cv2.imwrite(os.path.join(SAVED_IMAGES, file_name), final_image)
    
    IPython.display.display(PIL.Image.fromarray(final_image))
    print("image saved")

# save_image(1,1)


## Train

In [ ]:
def train():
    for epoch in range(1, EPOCHS):
        steps = 0
        test = None
        while True:
            mask, original = data.get_data(BATCH)
            if mask is None or original is None:
                break
            mask = mask / 127.5 - 1
            batch_size = mask.shape[0]

            mask_image, missing_image = mask_black(original)
            mask_image = mask_image / 127.5 - 1
            missing_image = missing_image / 127.5 - 1

            # Discriminator
            gen_missing = GEN.predict(mask_image)

            real = np.ones([batch_size, 1])
            fake = np.zeros([batch_size, 1])

            d_loss_original = DCRM.train_on_batch(missing_image, real)
            d_loss_mask = DCRM.train_on_batch(gen_missing, fake)
            d_loss = 0.5 * np.add(d_loss_original, d_loss_mask)

            # Generator
            for i in range(2):
                g_loss = COMBINED.train_on_batch(mask_image, [real, missing_image])
            log = "epoch: %d, steps: %d, DIS: %5f, GEN: %s" %(epoch, steps, d_loss[0], str(g_loss))
            print(log)
            # Writing to log to 'log.txt'
            with open('log.txt', 'a') as f:
                f.write("%s\n"%log)

            steps += 1

        if (epoch) % 10 == 0:
            save_model()
            save_image(epoch, steps)